# Final Predictions
Generate predictions on the test set using the best-performing model.

In [ ]:
import os, sys
import joblib
import pandas as pd
sys.path.append('..')
from src.utils.helpers import create_features
from src.models.ensemble import load_ensemble
from src.data_processing.load_data import save_csv_data
from src.evaluation.metrics import nrmse
config_path = '../config.yaml'
assert os.path.exists(config_path), 'Missing config.yaml'
config = __import__('yaml').safe_load(open(config_path))
test_path = config['data']['raw']['test']
features_path = config['data']['processed']['features_test']
model_path = os.path.join(config['models']['trained_models'], 'best_model.pkl')
ensemble_path = os.path.join(config['models']['trained_models'], 'ensemble_model.pkl')


## Feature Engineering on Test Data

In [ ]:
test_features = create_features(test_path, config['feature_engineering']['timestamp_col'],
                             config['feature_engineering']['cols_to_lag'],
                             config['feature_engineering']['window_sizes'])
save_csv_data(test_features, features_path)
test_features.head()

## Load Model and Predict

In [ ]:
model = joblib.load(model_path) if os.path.exists(model_path) else None
if model is None:
    raise FileNotFoundError('Trained model not found')
preds = model.predict(test_features.select_dtypes(include=['number']))
submission = pd.DataFrame({'prediction': preds})
save_csv_data(submission, os.path.join(config['data']['submissions'], 'submission_final.csv'))
submission.head()

## Ensemble Prediction (Optional)

In [ ]:
if os.path.exists(ensemble_path):
    ensemble = load_ensemble(ensemble_path)
    ensemble_preds = ensemble.predict(test_features.select_dtypes(include=['number']))
    ensemble_df = pd.DataFrame({'prediction': ensemble_preds})
    save_csv_data(ensemble_df, os.path.join(config['data']['submissions'], 'submission_ensemble.csv'))
    ensemble_df.head()
else:
    print('No ensemble model found.')